![dvd_image](dvd_image.jpg)

A DVD rental company needs your help! They want to figure out how many days a customer will rent a DVD for based on some features and has approached you for help. They want you to try out some regression models which will help predict the number of days a customer will rent a DVD for. The company wants a model which yeilds a MSE of 3 or less on a test set. The model you make will help the company become more efficient inventory planning.

The data they provided is in the csv file `rental_info.csv`. It has the following features:
- `"rental_date"`: The date (and time) the customer rents the DVD.
- `"return_date"`: The date (and time) the customer returns the DVD.
- `"amount"`: The amount paid by the customer for renting the DVD.
- `"amount_2"`: The square of `"amount"`.
- `"rental_rate"`: The rate at which the DVD is rented for.
- `"rental_rate_2"`: The square of `"rental_rate"`.
- `"release_year"`: The year the movie being rented was released.
- `"length"`: Lenght of the movie being rented, in minuites.
- `"length_2"`: The square of `"length"`.
- `"replacement_cost"`: The amount it will cost the company to replace the DVD.
- `"special_features"`: Any special features, for example trailers/deleted scenes that the DVD also has.
- `"NC-17"`, `"PG"`, `"PG-13"`, `"R"`: These columns are dummy variables of the rating of the movie. It takes the value 1 if the move is rated as the column name and 0 otherwise. For your convinience, the reference dummy has already been dropped.

In [9]:
# Importing required packages
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Lasso
import matplotlib.pyplot as plt

# Loading dataset
rental_df = pd.read_csv("rental_info.csv", parse_dates = ["rental_date", "return_date"])

# Adding new columns
rental_df["rental_length_days"] = (rental_df["return_date"]- rental_df["rental_date"]).dt.days
rental_df["deleted_scenes"] = np.where(rental_df["special_features"].str.contains("Deleted Scenes"), 1, 0)
rental_df["behind_the_scenes"] = np.where(rental_df["special_features"].str.contains("Behind the Scenes"), 1, 0)

# Creating features and targets
X = rental_df.drop(["rental_date", "return_date", "special_features", "rental_length_days"], axis = 1)
y = rental_df["rental_length_days"]

# Training and testing model
scores = []
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 9)
val = [0.001, 0.01, 0.1, 1, 10, 20, 50]
for alpha in val:
    lasso = Lasso(alpha = alpha)
    lasso.fit(X_train, y_train)
    scores.append(lasso.score(X_test, y_test))
alpha_score = zip(val, scores)
print(dict(alpha_score))

{0.001: 0.5856138256624186, 0.01: 0.584446499685334, 0.1: 0.5607287246321488, 1: 0.46395512311674025, 10: 0.2804205154084213, 20: 0.19766488247813552, 50: -5.482979101345542e-05}


In [11]:
# Model fitting and testing
alpha = 0.01
lasso = Lasso(alpha = alpha)
lasso.fit(X_train, y_train)
y_pred = lasso.predict(X_test)
MSE = mean_squared_error(y_test, y_pred)
print(MSE)

2.950251383377225
